In [44]:
import org.apache.spark.sql.SparkSession

lastException: Throwable = null


In [45]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@147c65c5


org.apache.spark.sql.SparkSession@147c65c5

In [46]:
spark.sql("use raj")

[]

In [5]:
spark.sql("show tables").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|     raj|             dep|      false|
|     raj|        dep_join|      false|
|     raj|             emp|      false|
|     raj|        emp_join|      false|
|     raj|        emp_part|      false|
|     raj|     emp_raj_par|      false|
|     raj|           join1|      false|
|     raj|           join2|      false|
|     raj|          joins1|      false|
|     raj|         joins_1|      false|
|     raj|          joins2|      false|
|     raj|          joins3|      false|
|     raj|      order_immu|      false|
|     raj|   orderitem_nor|      false|
|     raj|   orderitem_orc|      false|
|     raj|   orderitem_par|      false|
|     raj|     order_items|      false|
|     raj|order_items_part|      false|
|     raj|        order_oi|      false|
|     raj|   order_oi_cast|      false|
+--------+----------------+-----------+
only showing top 20 rows



In [12]:
spark.sql("describe formatted raj.order_immu").show(false)

+----------------------------+---------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                              |comment|
+----------------------------+---------------------------------------------------------------------------------------+-------+
|order_id                    |int                                                                                    |null   |
|order_date                  |timestamp                                                                              |null   |
|order_product_id            |int                                                                                    |null   |
|order_status                |string                                                                                 |null   |
|                            |                                                                                 

In [47]:
import org.apache.spark.sql.functions._

In [48]:
import spark.implicits._

In [49]:
import org.apache.spark.sql.Column

In [50]:
val order=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders/part-00000-6502fdf1-89f6-48b4-ae4c-6c0376ab98ce-c000.snappy.parquet")

order = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [14]:
order.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- Order_date: timestamp (nullable = true)
 |-- Order_product_id: integer (nullable = true)
 |-- Order_status: string (nullable = true)



In [15]:
order.show()

+--------+-------------------+----------------+---------------+
|order_id|         Order_date|Order_product_id|   Order_status|
+--------+-------------------+----------------+---------------+
|       1|2013-07-25 00:00:00|           11599|         CLOSED|
|       2|2013-07-25 00:00:00|             256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|           12111|       COMPLETE|
|       4|2013-07-25 00:00:00|            8827|         CLOSED|
|       5|2013-07-25 00:00:00|           11318|       COMPLETE|
|       6|2013-07-25 00:00:00|            7130|       COMPLETE|
|       7|2013-07-25 00:00:00|            4530|       COMPLETE|
|       8|2013-07-25 00:00:00|            2911|     PROCESSING|
|       9|2013-07-25 00:00:00|            5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|            5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|             918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|            1837|         CLOSED|
|      13|2013-07-25 00:00:00|          

In [17]:
order.filter(col("Order_date") like "2014-07%").show()

+--------+-------------------+----------------+---------------+
|order_id|         Order_date|Order_product_id|   Order_status|
+--------+-------------------+----------------+---------------+
|   54004|2014-07-01 00:00:00|            7079|       COMPLETE|
|   54005|2014-07-01 00:00:00|            3867|PENDING_PAYMENT|
|   54006|2014-07-01 00:00:00|            6547|PENDING_PAYMENT|
|   54007|2014-07-01 00:00:00|           11772|         CLOSED|
|   54008|2014-07-01 00:00:00|           10453|PENDING_PAYMENT|
|   54009|2014-07-01 00:00:00|            3971|       COMPLETE|
|   54010|2014-07-01 00:00:00|            4812|       COMPLETE|
|   54011|2014-07-01 00:00:00|            3933|        PENDING|
|   54012|2014-07-01 00:00:00|           10070|        PENDING|
|   54013|2014-07-01 00:00:00|            3021|        PENDING|
|   54014|2014-07-01 00:00:00|            3666|       CANCELED|
|   54015|2014-07-01 00:00:00|           11402|       COMPLETE|
|   54016|2014-07-01 00:00:00|          

In [19]:
order.filter(col("Order_date") like "2014-07%").groupBy(col("Order_status")).count().show()

+---------------+-----+
|   Order_status|count|
+---------------+-----+
|PENDING_PAYMENT|  979|
|       COMPLETE| 1419|
|        ON_HOLD|  228|
| PAYMENT_REVIEW|   54|
|     PROCESSING|  561|
|         CLOSED|  504|
|SUSPECTED_FRAUD|  101|
|        PENDING|  517|
|       CANCELED|  105|
+---------------+-----+



In [51]:
val orderItem=spark.read.schema("order_item_id int,order_item_order_id int,order_item_product_id int,order_item_quantity int,order_item_subtotal decimal(17,4),order_item_product_price decimal(17,4)").csv("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_item_new")

orderItem = [order_item_id: int, order_item_order_id: int ... 4 more fields]


[order_item_id: int, order_item_order_id: int ... 4 more fields]

In [21]:
orderItem.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|           299.9800|                299.9800|
|            2|                  2|                 1073|                  1|           199.9900|                199.9900|
|            3|                  2|                  502|                  5|           250.0000|                 50.0000|
|            4|                  2|                  403|                  1|           129.9900|                129.9900|
|            5|                  4|                  897|                  2|            49.9800|                 24.9900|
|            6| 

In [24]:
orderItem.groupBy("order_item_order_id").sum("order_item_subtotal").orderBy("order_item_order_id").show()

+-------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|
+-------------------+------------------------+
|                  1|                299.9800|
|                  2|                579.9800|
|                  4|                699.8500|
|                  5|               1129.8600|
|                  7|                579.9200|
|                  8|                729.8400|
|                  9|                599.9600|
|                 10|                651.9200|
|                 11|                919.7900|
|                 12|               1299.8700|
|                 13|                127.9600|
|                 14|                549.9400|
|                 15|                925.9100|
|                 16|                419.9300|
|                 17|                694.8400|
|                 18|                449.9600|
|                 19|                699.9600|
|                 20|                879.8600|
|            

In [28]:
orderItem.groupBy("order_item_order_id").agg(round(sum("order_item_subtotal"),2) alias "total revenue").orderBy("order_item_order_id").show()

+-------------------+-------------+
|order_item_order_id|total revenue|
+-------------------+-------------+
|                  1|       299.98|
|                  2|       579.98|
|                  4|       699.85|
|                  5|      1129.86|
|                  7|       579.92|
|                  8|       729.84|
|                  9|       599.96|
|                 10|       651.92|
|                 11|       919.79|
|                 12|      1299.87|
|                 13|       127.96|
|                 14|       549.94|
|                 15|       925.91|
|                 16|       419.93|
|                 17|       694.84|
|                 18|       449.96|
|                 19|       699.96|
|                 20|       879.86|
|                 21|       372.91|
|                 23|       299.98|
+-------------------+-------------+
only showing top 20 rows



In [52]:
order.createTempView("orders")

In [31]:
spark.sql("show tables").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|     raj|             dep|      false|
|     raj|        dep_join|      false|
|     raj|             emp|      false|
|     raj|        emp_join|      false|
|     raj|        emp_part|      false|
|     raj|     emp_raj_par|      false|
|     raj|           join1|      false|
|     raj|           join2|      false|
|     raj|          joins1|      false|
|     raj|         joins_1|      false|
|     raj|          joins2|      false|
|     raj|          joins3|      false|
|     raj|      order_immu|      false|
|     raj|   orderitem_nor|      false|
|     raj|   orderitem_orc|      false|
|     raj|   orderitem_par|      false|
|     raj|     order_items|      false|
|     raj|order_items_part|      false|
|     raj|        order_oi|      false|
|     raj|   order_oi_cast|      false|
+--------+----------------+-----------+
only showing top 20 rows



In [37]:
spark.sql("select * from orders where Order_date like '2014-07%' ").show()

+--------+-------------------+----------------+---------------+
|order_id|         Order_date|Order_product_id|   Order_status|
+--------+-------------------+----------------+---------------+
|   54004|2014-07-01 00:00:00|            7079|       COMPLETE|
|   54005|2014-07-01 00:00:00|            3867|PENDING_PAYMENT|
|   54006|2014-07-01 00:00:00|            6547|PENDING_PAYMENT|
|   54007|2014-07-01 00:00:00|           11772|         CLOSED|
|   54008|2014-07-01 00:00:00|           10453|PENDING_PAYMENT|
|   54009|2014-07-01 00:00:00|            3971|       COMPLETE|
|   54010|2014-07-01 00:00:00|            4812|       COMPLETE|
|   54011|2014-07-01 00:00:00|            3933|        PENDING|
|   54012|2014-07-01 00:00:00|           10070|        PENDING|
|   54013|2014-07-01 00:00:00|            3021|        PENDING|
|   54014|2014-07-01 00:00:00|            3666|       CANCELED|
|   54015|2014-07-01 00:00:00|           11402|       COMPLETE|
|   54016|2014-07-01 00:00:00|          

In [53]:
spark.sql("select Order_status,count(1) from orders where Order_date like '2014-01%' group by Order_status").show()

+---------------+--------+
|   Order_status|count(1)|
+---------------+--------+
|PENDING_PAYMENT|    1334|
|       COMPLETE|    1911|
|        ON_HOLD|     365|
| PAYMENT_REVIEW|      77|
|     PROCESSING|     712|
|         CLOSED|     633|
|SUSPECTED_FRAUD|     131|
|        PENDING|     635|
|       CANCELED|     110|
+---------------+--------+



In [55]:
orderItem.createTempView("orderitem")

In [56]:
spark.sql("select * from orderitem").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|           299.9800|                299.9800|
|            2|                  2|                 1073|                  1|           199.9900|                199.9900|
|            3|                  2|                  502|                  5|           250.0000|                 50.0000|
|            4|                  2|                  403|                  1|           129.9900|                129.9900|
|            5|                  4|                  897|                  2|            49.9800|                 24.9900|
|            6| 

In [63]:
spark.sql("select order_item_order_id,round(sum(order_item_subtotal),2) from orderitem group by order_item_order_id order by order_item_order_id").show()

+-------------------+----------------------------------+
|order_item_order_id|round(sum(order_item_subtotal), 2)|
+-------------------+----------------------------------+
|                  1|                            299.98|
|                  2|                            579.98|
|                  4|                            699.85|
|                  5|                           1129.86|
|                  7|                            579.92|
|                  8|                            729.84|
|                  9|                            599.96|
|                 10|                            651.92|
|                 11|                            919.79|
|                 12|                           1299.87|
|                 13|                            127.96|
|                 14|                            549.94|
|                 15|                            925.91|
|                 16|                            419.93|
|                 17|          

In [66]:
orderItem.write.saveAsTable("OrderItems")

lastException: Throwable = null
